# Kaggle

In [1]:
import boto3
from botocore.client import Config
from datetime import datetime
import os
from kaggle.api.kaggle_api_extended import KaggleApi
import tempfile
import zipfile
import warnings
import logging

#Desativar os warnings do boto3
warnings.filterwarnings("ignore", category=UserWarning, module="urllib3")
logging.getLogger('urllib3').setLevel(logging.ERROR)

In [2]:
def LeituraAPI_direto_minio(folder_name: str, source: str):
    """
    Baixa dataset do Kaggle diretamente para o MinIO (camada Land)
    """
    try:
        # Autenticação Kaggle
        os.environ['KAGGLE_CONFIG_DIR'] = '/user/.kaggle'  # Corrigido o caminho
        api = KaggleApi()
        api.authenticate()
        print("✅ Autenticação Kaggle bem-sucedida")

        # Configuração MinIO
        minio_client = boto3.client(
            's3',
            endpoint_url='http://minio:9000',
            aws_access_key_id='minio',
            aws_secret_access_key='minio123',
            config=Config(signature_version='s3v4'),
            region_name='us-east-1'
        )

        # Bucket para camada Land
        bucket_name = 'land'

        # Verifica/Cria bucket
        try:
            minio_client.head_bucket(Bucket=bucket_name)
            print(f"✅ Bucket '{bucket_name}' existe")
        except:
            minio_client.create_bucket(Bucket=bucket_name)
            print(f"✅ Bucket '{bucket_name}' criado")

        # Cria estrutura de pastas no MinIO
        s3_base_path = f"{folder_name}/{datetime.now().strftime('%m_%Y')}"
        print(f"📁 Estrutura MinIO: {s3_base_path}")

        # Cria diretório temporário
        with tempfile.TemporaryDirectory() as tmp_dir:
            print(f"📥 Baixando dataset {source} para diretório temporário...")
            
            # Baixa o dataset
            api.dataset_download_files(source, path=tmp_dir, unzip=True)
            
            # Percorre todos os arquivos baixados
            for root, dirs, files in os.walk(tmp_dir):
                for file in files:
                    local_file_path = os.path.join(root, file)
                    
                    # Calcula o caminho relativo para manter a estrutura
                    relative_path = os.path.relpath(local_file_path, tmp_dir)
                    s3_key = f"{s3_base_path}/{relative_path}"
                    
                    # Cria diretório no MinIO se necessário
                    s3_dir = os.path.dirname(s3_key)
                    if s3_dir:
                        try:
                            minio_client.put_object(
                                Bucket=bucket_name,
                                Key=s3_dir + '/',
                                Body=b''
                            )
                        except:
                            pass  # Diretório já pode existir
                    
                    # Faz upload do arquivo
                    print(f"⬆️  Enviando {file} para MinIO...")
                    minio_client.upload_file(
                        Filename=local_file_path,
                        Bucket=bucket_name,
                        Key=s3_key
                    )
                    print(f"✅ Upload concluído: {s3_key}")

        print(f"🎉 Download e upload concluídos para bucket '{bucket_name}'")
        return True

    except Exception as e:
        print(f"❌ Erro: {e}")
        import traceback
        traceback.print_exc()
        return False


In [3]:
folder_name = 'uber_dataset'
source = 'fivethirtyeight/uber-pickups-in-new-york-city'

LeituraAPI_direto_minio(folder_name=folder_name, source=source)

✅ Autenticação Kaggle bem-sucedida
✅ Bucket 'land' existe
📁 Estrutura MinIO: uber_dataset/09_2025
📥 Baixando dataset fivethirtyeight/uber-pickups-in-new-york-city para diretório temporário...
Dataset URL: https://www.kaggle.com/datasets/fivethirtyeight/uber-pickups-in-new-york-city


2025-09-05 17:28:37,306 WARNING Failed to parse headers (url=http://minio:9000/land/uber_dataset/09_2025/other-American_B01362.csv): [MissingHeaderBodySeparatorDefect()], unparsed data: 'HTTP/1.1 200 OK\r\nAccept-Ranges: bytes\r\nContent-Length: 0\r\nETag: "7fa8cbfc4976c493e5c9402dafc8c579"\r\nServer: MinIO\r\nStrict-Transport-Security: max-age=31536000; includeSubDomains\r\nVary: Origin\r\nVary: Accept-Encoding\r\nX-Amz-Checksum-Crc32: b/JeXg==\r\nX-Amz-Id-2: dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8\r\nX-Amz-Request-Id: 1862731004ED563F\r\nX-Content-Type-Options: nosniff\r\nX-Xss-Protection: 1; mode=block\r\nDate: Fri, 05 Sep 2025 17:28:37 GMT\r\n\r\n'
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/urllib3/connectionpool.py", line 469, in _make_request
    assert_header_parsing(httplib_response.msg)
  File "/opt/conda/lib/python3.10/site-packages/urllib3/util/response.py", line 91, in assert_header_parsing
    raise HeaderPar

⬆️  Enviando other-American_B01362.csv para MinIO...


KeyboardInterrupt: 